In [ ]:
#Instalando a biblioteca no Google Gemin
!pip install -q -U google-generativeai

#Importando bibliotecas do Google Gemin, mumpy e pandas
import numpy as np
import pandas as pd
import google.generativeai as gemini

#Obtendo dados do usuário
from google.colab import userdata

#Armazenando a chave (API_KEY) em uma variável do colab
api_key = userdata.get('SECRET_KEY')

#Atribuindo a chave às configurações do gemini
gemini.configure(api_key=api_key)

for m in gemini.list_models():
	if 'embedContent' in m.supported_generation_methods:
		print(m.name)

#Gerando o primeiro Embed (Contexto)
text = "Hello world"
result = gemini.embed_content(model="models/embedding-001", content=text)
print(result['embedding'])
print(len(result['embedding']))

#Gerando o segundo Embed (Contexto)
text = [
    "What is the meaning of life?",
    "How much wood would a woodchuck chuck?",
    "How does the brain work?"]
result = gemini.embed_content(model="models/embedding-001", content=text)
for embedding in result['embedding']:
  print(str(embedding)[:50], "...TRIMMED")

#Gerando o terceiro Embed (Contexto)
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemin API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativas")

embedding = gemini.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embedding)

#Listagem de documentos
document1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar"}

document2 = {
    "Título": "TouchScreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso à internet"}

document3 = {
    "Título": "Mudança de marcha",
    "Conteúdo": "O Googlecar tem um transmissão automática. Para trocar as marchas."}

documents = [document1, document2, document3]

df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']

model = "models/embedding-001"

def embed_fn(title, text):
  return gemini.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

#Gerando função para retornar conteudo do Embedding criando anteriormente
def consultar_documento(consulta, base, model):
  embedding = gemini.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding)
  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

consulta = "Como trocar marcha?"
resultado = consultar_documento(consulta, df, model)
print(resultado)

#Mesclando o modelo criado acima (embaddings) com o modelo existente (gemini)
prompt = f"Reescreva esse texto de uma forma mais criativa: {resultado}"
generation_conf = {
    "temperature": 0.5,
    "candidate_count": 1,
}
model2 = gemini.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_conf)
response = model2.generate_content(prompt)
print(response.text)
